In [173]:
from pprint import pprint
import random

In [174]:
def create_user_interest_dict(*interests):
    interest_count = len(interests)
    interest_weight = round((100-10) / interest_count,2) # Subtract 10 for 'Random'
    user_interests = {interest: interest_weight for interest in interests}
    user_interests['Random'] = 10 # Add 'Random' with constant value of 10
    return user_interests

carl_interests = create_user_interest_dict("DevOps", "Tests (SAT)", "Machine Learning", "Cats", "Cooking")
pprint(carl_interests)

{'Cats': 18.0,
 'Cooking': 18.0,
 'DevOps': 18.0,
 'Machine Learning': 18.0,
 'Random': 10,
 'Tests (SAT)': 18.0}


In [180]:
def observeActionsTaken(video_tag, liked, shared, watched, loop_count):
    # Actions and their corresponding points to add
    Actions = {
        "Like": 0.1,
        "Share": 0.2,
        "Watch": 0.1,  # more than 50% of total duration
        # todo: experiment with increasing watch value
        "Loop": 0.05,
    }

    if liked:
        total_points += Actions["Like"]
    if shared:
        total_points += Actions["Share"]
    if watched:
        total_points += Actions["Watch"]
    total_points += loop_count * Actions["Loop"]

    print(f"\nUpdating weights for video tag: {video_tag}")
    print(f"Total points to add: {total_points}")

    return total_points

In [176]:
def handleUpdateScore(user_dict_interest, video_tag, total_points):

    # Increase the weight of the interest based on the action points
    user_dict_interest[video_tag] += total_points
    print(f"Updated {video_tag} to {user_dict_interest[video_tag]}\n")

    # Deduct the total points proportionally from existing interests (excluding 'Random' and the video_tag itself)
    total_deductable_weight = sum(
        weight for interest, weight in user_dict_interest.items()
        if interest != "Random" and interest != video_tag

    )
    print(f"Total weight to deduct from other interests: {total_points}\nTotal deductable weight (excluding 'Random' and {video_tag}): {total_deductable_weight}")

    for interest in user_dict_interest:
        if interest != "Random" and interest != video_tag:
            deduction = (total_points / total_deductable_weight) * user_dict_interest[interest]
            user_dict_interest[interest] -= deduction

            print(f"Deducted {deduction} from {interest}, new weight: {user_dict_interest[interest]}")

    return user_dict_interest

In [193]:
def handleAddNewTag(user_dict_interest, video_tag):
    # Add new interest if it doesn't exist and give initial points
        initial_points = 5
        user_dict_interest[video_tag] = initial_points
        print(f"Added new interest {video_tag} with initial weight: {initial_points}")

        # Deduct initial points proportionally from existing interests (excluding 'Random')
        total_deductable_weight = sum(
            weight
            for interest, weight in user_dict_interest.items()
            if interest != "Random" and interest != video_tag
        )

        print(f"Total weight to deduct from other interests: {initial_points}\nTotal deductable weight (excluding 'Random' and {video_tag}): {total_deductable_weight}")

        for interest in user_dict_interest:
            if interest != "Random" and interest != video_tag:
                deduction = (
                    initial_points / total_deductable_weight
                ) * user_dict_interest[interest]
                user_dict_interest[interest] -= deduction

                pprint(f"Deducted {deduction} from {interest}, new weight: {user_dict_interest[interest]}")

        return user_dict_interest

In [194]:
def update_weight(
    user_dict_interest,
    video_tag,
    liked=False,
    shared=False,
    watched=False,
    loop_count=0,
):

    total_points = observeActionsTaken(video_tag, liked, shared, watched, loop_count)

    if video_tag in user_dict_interest:
        handleUpdateScore(user_dict_interest, video_tag, total_points)
    else:
        handleAddNewTag(user_dict_interest, video_tag)

    # Check total weight to ensure it sums to 100 (for debugging purposes)
    total_weight = sum(user_dict_interest.values())
    print(f"Total weight after update: {total_weight} (should be 100)\n")
    print(f"Updated interests: {user_dict_interest}\n")

    return user_dict_interest

In [196]:
# Test the update_weight function
print("Before update:", carl_interests)
update_weight(carl_interests, "DevOps", liked=True, watched=True, loop_count=2)
print("After update:", carl_interests)

Before update: {'DevOps': 19.200000000000003, 'Tests (SAT)': 17.700000000000003, 'Machine Learning': 17.700000000000003, 'Cats': 17.700000000000003, 'Cooking': 17.700000000000003, 'Random': 10}

Updating weights for video tag: DevOps
Total points to add: 0.30000000000000004
Updated DevOps to 19.500000000000004

Total weight to deduct from other interests: 0.30000000000000004
Total deductable weight (excluding 'Random' and DevOps): 70.80000000000001
Deducted 0.07500000000000001 from Tests (SAT), new weight: 17.625000000000004
Deducted 0.07500000000000001 from Machine Learning, new weight: 17.625000000000004
Deducted 0.07500000000000001 from Cats, new weight: 17.625000000000004
Deducted 0.07500000000000001 from Cooking, new weight: 17.625000000000004
Total weight after update: 100.00000000000001 (should be 100)

Updated interests: {'DevOps': 19.500000000000004, 'Tests (SAT)': 17.625000000000004, 'Machine Learning': 17.625000000000004, 'Cats': 17.625000000000004, 'Cooking': 17.62500000000

In [192]:
def get_random_interest(user_dict_interest):
    random_value = random.uniform(0, 100)
    print("_"*50)
    print(f"\nRandom value: {random_value}\n")
    cumulative_probability = 0.0
    visualization = []

    for interest, probability in user_dict_interest.items():
        previous_cumulative_probability = cumulative_probability
        cumulative_probability += probability
        visualization.append(
            (interest, previous_cumulative_probability, cumulative_probability)
        )
        pprint(
            f"Checking interest: {interest}, cumulative range: {previous_cumulative_probability} - {cumulative_probability}"
        )
        if random_value <= cumulative_probability:
            print(f"Selected interest: {interest}\n")
            break

    return interest, visualization

In [ ]:
# Test the get_random_interest function
selected_interest, viz = get_random_interest(carl_interests)
print(f"Randomly selected interest: {selected_interest}")
print(viz)

__________________________________________________

Random value: 40.79202542856118

'Checking interest: DevOps, cumulative range: 0.0 - 22.8'
'Checking interest: Machine Learning, cumulative range: 22.8 - 45.2'
Selected interest: Machine Learning

Randomly selected interest: Machine Learning
[('DevOps', 0.0, 22.8), ('Machine Learning', 22.8, 45.2)]


___

# ARCHIVE

In [ ]:
# class User
    # def like


In [ ]:
# Init - Choose Categories

# Carl chose 2 categories
# Always adds up to 100
CarlInterests = {
    'DevOps': 45,
    'Machine Learning': 45,
    'Random': 10,
}

print(CarlInterests)

{'DevOps': 45, 'Machine Learning': 45, 'Random': 10}


In [ ]:
# Updating Weights

# Is called when watching a video

# Actions taken by user and their corresponding points to add
Actions = {
    'Like': 0.1,
    'Share': 0.2,
    'Watch': 0.1, # more than 50% of total duration
    'Loop': 0.05
}

tag = "DevOps"

def update_weight(user_dict_interest, video_tag, liked=False, shared=False, watched=False,):
    if video_tag in user_dict_interest:
        print(f"{video_tag} exists in {user_dict_interest}\n")

        print("Recalculating...")
        print(Actions)
        user_dict_interest[tag] = user_dict_interest.get(tag)+Actions.get("Like")
        print(f"{user_dict_interest.get(tag)}\n")

        for interest, weight in user_dict_interest.items():
            if interest != video_tag and interest != 'Random':
                  print(interest, weight)
                  user_dict_interest[interest] = weight - Actions.get("Like")
                  print(interest, weight)

    # else:

update_weight(CarlInterests, "DevOps")

print(CarlInterests)


DevOps exists in {'DevOps': 45, 'Machine Learning': 45, 'Random': 10}

Recalculating...
{'Like': 0.1, 'Share': 0.2, 'Watch': 0.1, 'Loop': 0.05}
45.1

Machine Learning 45
Machine Learning 45
{'DevOps': 45.1, 'Machine Learning': 44.9, 'Random': 10}


In [ ]:
# Auto-Adding Mechanism

In [ ]:
def update_weight(
    user_interests, video_tag, liked=False, shared=False, watched=False, loopCount=0
):
    user_actions = {
        "Like": liked,
        "Share": shared,
        "Watch": watched,
        "Loop": loopCount > 0,
    }
    for action, performed in user_actions.items():
        if performed:
            user_interests[video_tag] = user_interests.get(video_tag, 0) + Actions.get(
                action, 0
            )
True /10

0.1